In [8]:
import pandas as pd
import pyarrow as pa

from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    NestedField,
    IntegerType,
    DoubleType,
    StringType,
    BooleanType,
    TimestampType,
)
from pyiceberg import Config

ImportError: cannot import name 'Config' from 'pyiceberg' (/home/ryano/werk/odp_lab/.venv/lib/python3.12/site-packages/pyiceberg/__init__.py)

In [6]:
import pyiceberg
print(pyiceberg.__version__)

0.10.0


In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

from pyiceberg.catalog import load_catalog
from pyiceberg.schema import Schema
from pyiceberg.types import (
    NestedField, IntegerType, LongType, DoubleType, StringType, BooleanType, DateType, TimestampType
)

# 1. Read CSV
df = pd.read_csv("Sample-Superstore.csv", encoding="latin-1", parse_dates=["Order Date", "Ship Date"])
# df = pd.read_csv("Sample-Superstore.csv", encoding="latin-1")


print(df)

      Row ID        Order ID Order Date  Ship Date       Ship Mode  \
0          1  CA-2016-152156 2016-11-08 2016-11-11    Second Class   
1          2  CA-2016-152156 2016-11-08 2016-11-11    Second Class   
2          3  CA-2016-138688 2016-06-12 2016-06-16    Second Class   
3          4  US-2015-108966 2015-10-11 2015-10-18  Standard Class   
4          5  US-2015-108966 2015-10-11 2015-10-18  Standard Class   
...      ...             ...        ...        ...             ...   
9989    9990  CA-2014-110422 2014-01-21 2014-01-23    Second Class   
9990    9991  CA-2017-121258 2017-02-26 2017-03-03  Standard Class   
9991    9992  CA-2017-121258 2017-02-26 2017-03-03  Standard Class   
9992    9993  CA-2017-121258 2017-02-26 2017-03-03  Standard Class   
9993    9994  CA-2017-119914 2017-05-04 2017-05-09    Second Class   

     Customer ID     Customer Name    Segment        Country             City  \
0       CG-12520       Claire Gute   Consumer  United States        Henderson 

In [2]:
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])
df["Postal Code"] = df["Postal Code"].astype(str)
print(df.dtypes)

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code              object
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
dtype: object


In [3]:
def pandas_dtype_to_iceberg(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return IntegerType()
    elif pd.api.types.is_float_dtype(dtype):
        return DoubleType()
    elif pd.api.types.is_bool_dtype(dtype):
        return BooleanType()
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return TimestampType()
    else:
        return StringType()
    
def dataframe_to_iceberg_schema(df: pd.DataFrame) -> Schema:
    fields = []
    field_id = 1

    for col, dtype in df.dtypes.items():
        iceberg_type = pandas_dtype_to_iceberg(dtype)

        fields.append(
            NestedField(
                field_id=field_id,
                name=col.lower().replace(" ", "_").replace("-", "_"),
                field_type=iceberg_type,
                required=False
            )
        )
        field_id += 1

    return Schema(*fields)

In [4]:
schema = dataframe_to_iceberg_schema(df)
print(schema)

table {
  1: row_id: optional int
  2: order_id: optional string
  3: order_date: optional timestamp
  4: ship_date: optional timestamp
  5: ship_mode: optional string
  6: customer_id: optional string
  7: customer_name: optional string
  8: segment: optional string
  9: country: optional string
  10: city: optional string
  11: state: optional string
  12: postal_code: optional string
  13: region: optional string
  14: product_id: optional string
  15: category: optional string
  16: sub_category: optional string
  17: product_name: optional string
  18: sales: optional double
  19: quantity: optional int
  20: discount: optional double
  21: profit: optional double
}


In [ ]:
print(catalog.list_namespaces())

In [ ]:
catalog.create_namespace("bronze")

In [ ]:
catalog.create_table(
    identifier="bronze.orders",
    schema=schema,
    location="s3a://lakehouse/bronze/orders"
)

In [ ]:
table = pa.Table.from_pandas(df)

pq.write_table(
    table,
    "orders.parquet",
    compression="snappy"
)


In [ ]:
table = catalog.load_table("raw.orders")

In [ ]:
import pyarrow as pa

df.columns = (
    df.columns
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("-", "_")
)

arrow_table = pa.Table.from_pandas(df, preserve_index=False)

table.append(arrow_table)
